# Kinematic sample
This notebook compiles data from lenses with imaging data and integrated velocity dispersion measurements.

Author: Simon Birrer

In [1]:
from hierarc.LensPosterior.kin_constraints import KinConstraints
from hierarc.Util import ifu_util
from lenstronomy.Util import constants as const
from astropy.io import fits
import numpy as np
import pickle
import os
import csv
import matplotlib.pyplot as plt
%matplotlib inline

## Sample from SLACS (re-modelled by A. Shajib)

In [2]:
dir_path = os.getcwd()

# path to folder with IFU data
ifu_folder = 'IFU_data/Barnabe2011-kinmaps2'
path2ifu = os.path.join(dir_path, ifu_folder)

# BOSS spectra observational conditions, https://arxiv.org/pdf/0805.1931.pdf Section 6.3

# circular apperture with diameter 3 arc seconds and seeing approx 1.4 arc seconds
kwargs_aperture_slit = {'aperture_type': 'shell', 'r_in': 0, 'r_out': 3/2., 'center_ra': 0.0, 'center_dec': 0}
kwargs_seeing_slit = {'psf_type': 'GAUSSIAN', 'fwhm': 1.4}

# numerical settings (not needed if power-law profiles with Hernquist light distribution is computed)
kwargs_numerics_galkin = {'interpol_grid_num': 1000,  # numerical interpolation, should converge -> infinity
                          'log_integration': True,  # log or linear interpolation of surface brightness and mass models
                          'max_integrate': 100, 'min_integrate': 0.001}  # lower/upper bound of numerical integrals

num_sample_model_ifu = 200  # number of draws from the model to generate a kinematic prediction for the IFU (more needed for accurate covariance calculation)
num_sample_model_slit = 100  # number of draws from the model to generate a kinematic prediction for the slit data

# anisotropy model
anisotropy_model = 'OM' # 'OM', 'GOM' 

# population distribution in the power-law slope
# this distribution is assumed on lenses within the sample that do not have a reliably measured slope from imaging data

gamma_mean_pop, gamma_error_pop = 2.10, 0.16

# file containing different measurements of the stellar kinematics of the SLACS lenses
v_disp_file_name = 'vdisp_slacs_asb.fits'
v_disp_file = os.path.join(dir_path, v_disp_file_name)
HDU = fits.open(v_disp_file)
vel_disp_data = HDU[1].data

# file containing all the information about the sample including lens model parameters from imaging data
file_name_param_catalogue = 'slacs_all_params.csv'
file_name_param = os.path.join(dir_path, file_name_param_catalogue)
print(file_name_param)

process_ifu = True
process_slit = True


/Users/sibirrer/Science/Projects/TDCOSMO/Analysis/Hierarchical/SLACS_sample/slacs_all_params.csv


In [3]:

def vel_disp(name):
    """
    reads out the velocity dispersion and error from the fits file. Table from Yiping Shu with products by Adam Bolton
    """
    for lens in vel_disp_data:
        if name == lens['SDSS_NAME']:
            sigma = lens['VDISP_ASB']
            if sigma > 0:
                return lens['VDISP_ASB'], lens['VDISP_ASB_ERR']
            else:
                print('no measured dispersion from tabel' )
    print('name %s not in sample  - using SDSS standard reduction' % (name))
    return None, None


def dispersion_from_file(file_path, r_bins, fiber_scale):
    """
    velocity dispersion estimate from a file containing the data set of velocity dispersion, velocity, flux and errors

    :param file_path: path to fits file
    param r_bins: array, bins in radial directions
    :param fiber_scale: separation of the fibers (map pixels)
    :return: velocity dispersion in bins and their errors
    """
    hdul = fits.open(file_path)
    velocity = hdul[1].data
    dispersion = hdul[2].data
    surface_brightness = hdul[3].data
    sn_ration = hdul[4].data
    d_v_low = hdul[5].data
    d_v_high = hdul[6].data
    d_v_mean = (d_v_high - d_v_low) / 2
    d_sigma_low = hdul[7].data
    d_sigma_high = hdul[8].data
    d_sigma_mean = (d_sigma_high - d_sigma_low) / 2
    
    disp_tot, disp_error = ifu_util.binned_total(dispersion, weight_map_disp=1./d_sigma_mean**2, velocity_map=velocity, 
                              weight_map_v=1./d_v_mean**2, flux_map=surface_brightness, fiber_scale=fiber_scale, r_bins=r_bins)
    return disp_tot, disp_error



#def process_sample(file_name, a_ani_0, ani_param_array, num_sample=200):
def process_sample_slit(file_name):
    posterior_list = []

    with open(file_name, newline='') as myFile:
        reader = csv.DictReader(myFile)
        for row in reader:
            name = row['name']
            print(name)
            flag_imaging = float(row['flag_imaging'])
            if flag_imaging < 0:
                pass
            else:
                if flag_imaging == 1:  
                    gamma, gamma_error = float(row['gamma']), float(row['gamma_error'])
                else:
                    gamma, gamma_error = gamma_mean_pop, gamma_error_pop
                kwargs = {'z_lens': float(row['z_lens']), 'z_source': float(row['z_source']),
                         'r_eff': float(row['r_eff']), 'r_eff_error': float(row['r_eff_error']),
                         'theta_E': float(row['theta_E']), 'theta_E_error': float(row['theta_E_error']),
                         'gamma': gamma, 'gamma_error': gamma_error}
                kwargs_lens_light, MGE_light, kwargs_mge_light = None, False, None
                sigma_v, sigma_v_error_independent = vel_disp(name=name)
                if sigma_v is None:
                    sigma_v = float(row['sigma_v'])
                    sigma_v_error_independent = float(row['sigma_v_error'])
                sigma_v = [sigma_v]
                sigma_v_error_independent = [sigma_v_error_independent]

                kin_constraints = KinConstraints(sigma_v=sigma_v, 
                                 sigma_v_error_independent=sigma_v_error_independent, 
                                 sigma_v_error_covariant=0, 
                                 kwargs_aperture=kwargs_aperture_slit, kwargs_seeing=kwargs_seeing_slit, 
                                 kwargs_numerics_galkin=kwargs_numerics_galkin, 
                                 anisotropy_model=anisotropy_model, kwargs_lens_light=kwargs_lens_light, 
                                 lens_light_model_list=['HERNQUIST'], MGE_light=MGE_light,
                                 kwargs_mge_light=kwargs_mge_light, num_kin_sampling=2000, num_psf_sampling=100, **kwargs)

                kwargs_posterior = kin_constraints.hierarchy_configuration(num_sample_model=num_sample_model_slit)
                kwargs_posterior['flag_imaging'] = flag_imaging
                kwargs_posterior['name'] = name
                posterior_list.append(kwargs_posterior)
    return posterior_list



#def process_sample(file_name, a_ani_0, ani_param_array, num_sample=200):
def process_sample_ifu(file_name):
    posterior_list = []

    with open(file_name, newline='') as myFile:
        reader = csv.DictReader(myFile)
        for row in reader:
            
            if float(row['IFU']) == 1:
                flag_ifu = int(float(row['flag_ifu']))
                if flag_ifu == 1:
                    name = row['name']
                    flag_imaging = float(row['flag_imaging'])

                    # if the imaging data leads to a good lens model we use this model, otherwise we make use of the sample distribution
                    if flag_imaging == 1:  
                        gamma, gamma_error = float(row['gamma']), float(row['gamma_error'])
                    else:
                        gamma, gamma_error = gamma_mean_pop, gamma_error_pop
                    print(name)
                    ifu_file = os.path.join(path2ifu, str(row['ifu_file_name']))
                    print('read in IFU data from %s' %ifu_file)

                    z_lens = float(row['z_lens'])
                    r_bins = np.linspace(0, float(row['r_max']), float(row['n_bin']) + 1)
                    kwargs_aperture = {'aperture_type': 'IFU_shells', 'r_bins': r_bins,  'center_ra': 0, 'center_dec': 0}
                    kwargs_seeing = {'psf_type': 'GAUSSIAN', 'fwhm': float(row['psf_ifu'])}

                    kwargs = {'z_lens': float(row['z_lens']), 'z_source': float(row['z_source']),
                                       'r_eff': float(row['r_eff']), 'r_eff_error': float(row['r_eff_error']),
                                       'theta_E': float(row['theta_E']), 'theta_E_error': float(row['theta_E_error']),
                                       'gamma': gamma, 'gamma_error': gamma_error}
                    kwargs_lens_light, MGE_light, kwargs_mge_light = None, False, None
                    sigma_v_, sigma_v_error_independent_ = dispersion_from_file(ifu_file, r_bins, float(row['fiber_scale']))
                    sigma_v = sigma_v_[np.isfinite(sigma_v_)]
                    sigma_v_error_independent = sigma_v_error_independent_[np.isfinite(sigma_v_)] 

                    ifu_kin = KinConstraints(sigma_v=sigma_v, 
                                    sigma_v_error_independent=sigma_v_error_independent, 
                                    sigma_v_error_covariant=0, 
                                    kwargs_aperture=kwargs_aperture, kwargs_seeing=kwargs_seeing, 
                                    kwargs_numerics_galkin=kwargs_numerics_galkin, 
                                    anisotropy_model=anisotropy_model, kwargs_lens_light=kwargs_lens_light, 
                                    lens_light_model_list=['HERNQUIST'], MGE_light=MGE_light,
                                    kwargs_mge_light=kwargs_mge_light, num_kin_sampling=2000, num_psf_sampling=500, **kwargs)

                    kwargs_posterior = ifu_kin.hierarchy_configuration(num_sample_model=num_sample_model_ifu)
                    kwargs_posterior['flag_ifu'] = flag_ifu

                    kwargs_posterior['flag_imaging'] = flag_imaging
                    kwargs_posterior['name'] = name
                    posterior_list.append(kwargs_posterior)
    return posterior_list


In [4]:
if anisotropy_model == 'OM':
    file_name_slit = 'slacs_slit_om_processed.pkl'
    file_name_ifu = 'slacs_ifu_om_processed.pkl'
if anisotropy_model == 'GOM':
    file_name_slit = 'slacs_slit_gom_processed.pkl'
    file_name_ifu = 'slacs_ifu_gom_processed.pkl'


if process_slit is True:
    posterior_list_slit = process_sample_slit(file_name_param)

    file = open(file_name_slit, 'wb')
    pickle.dump(posterior_list_slit, file)
    file.close()

            
if process_ifu is True:
    posterior_list_ifu = process_sample_ifu(file_name_param)
    file = open(file_name_ifu, 'wb')
    pickle.dump(posterior_list_ifu, file)
    file.close()
  

SDSSJ1627-0053
SDSSJ2238-0754
SDSSJ2300+0022
SDSSJ2303+1422
SDSSJ0737+3216
SDSSJ1402+6321
SDSSJ1250+0523
SDSSJ1630+4520
SDSSJ0959+0410
SDSSJ0330-0020
SDSSJ0252+0039
SDSSJ0029-0055
SDSSJ0728+3835
SDSSJ2343-0030
SDSSJ1204+0358
SDSSJ0037-0942
SDSSJ0903+4116
SDSSJ1112+0826
SDSSJ1306+0600
SDSSJ1531-0105
SDSSJ1621+3931
SDSSJ1636+4707
SDSSJ1313+4615
SDSSJ0912+0029
SDSSJ1143-0144
SDSSJ1153+4612
SDSSJ2302-0840
SDSSJ2321-0939
SDSSJ0008-0004
SDSSJ0044+0113
SDSSJ0819+4534
SDSSJ0935-0003
SDSSJ0936+0913
SDSSJ0959+4416
SDSSJ1016+3859
SDSSJ1020+1122
SDSSJ1100+5329
SDSSJ1134+6027
SDSSJ1142+1001
SDSSJ1213+6708
SDSSJ1218+0830
SDSSJ1319+1504


/Users/sibirrer/Software/lenstronomy/lenstronomy/GalKin/galkin.py:109: ComplexWarning: Casting complex values to real discards the imaginary part
  sigma2_R_sum[ifu_index] += sigma2_R


SDSSJ1432+6317
SDSSJ1614+4522
SDSSJ1644+2625
SDSSJ2347-0005
no measured dispersion from tabel
name SDSSJ2347-0005 not in sample  - using SDSS standard reduction
SDSSJ1023+4230
SDSSJ1403+0006
SDSSJ1538+5817
SDSSJ2341+0000
SDSSJ0216-0813
SDSSJ1250-0135
SDSSJ1251-0209
name SDSSJ1251-0209 not in sample  - using SDSS standard reduction
SDSSJ1330-0148
SDSSJ1443+0304
SDSSJ1451-0239
SDSSJ1627-0053
read in IFU data from /Users/sibirrer/Science/Projects/TDCOSMO/Analysis/Hierarchical/SLACS_sample/IFU_data/Barnabe2011-kinmaps2/J1627_kinmap.fits
SDSSJ2300+0022
read in IFU data from /Users/sibirrer/Science/Projects/TDCOSMO/Analysis/Hierarchical/SLACS_sample/IFU_data/Barnabe2011-kinmaps2/J2300_kinmap.fits
SDSSJ2303+1422
read in IFU data from /Users/sibirrer/Science/Projects/TDCOSMO/Analysis/Hierarchical/SLACS_sample/IFU_data/Barnabe2011-kinmaps2/J2303_kinmap.fits


/Users/sibirrer/Library/Python/3.6/lib/python/site-packages/ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in true_divide


SDSSJ1250+0523
read in IFU data from /Users/sibirrer/Science/Projects/TDCOSMO/Analysis/Hierarchical/SLACS_sample/IFU_data/Barnabe2011-kinmaps2/J1250A_kinmap.fits
SDSSJ1204+0358
read in IFU data from /Users/sibirrer/Science/Projects/TDCOSMO/Analysis/Hierarchical/SLACS_sample/IFU_data/Barnabe2011-kinmaps2/J1204_kinmap.fits
SDSSJ0037-0942
read in IFU data from /Users/sibirrer/Science/Projects/TDCOSMO/Analysis/Hierarchical/SLACS_sample/IFU_data/Barnabe2011-kinmaps2/J0037_kinmap.fits
SDSSJ0912+0029
read in IFU data from /Users/sibirrer/Science/Projects/TDCOSMO/Analysis/Hierarchical/SLACS_sample/IFU_data/Barnabe2011-kinmaps2/J0912_kinmap.fits
SDSSJ2321-0939
read in IFU data from /Users/sibirrer/Science/Projects/TDCOSMO/Analysis/Hierarchical/SLACS_sample/IFU_data/Barnabe2011-kinmaps2/J2321_kinmap.fits
SDSSJ0935-0003
read in IFU data from /Users/sibirrer/Science/Projects/TDCOSMO/Analysis/Hierarchical/SLACS_sample/IFU_data/Barnabe2011-kinmaps2/J0935_kinmap.fits
SDSSJ0216-0813
read in IFU data f